In [2]:
# On your Windows client (VSCode)
import pyspark
print("PySpark version:", pyspark.__version__)

PySpark version: 3.3.2


In [3]:
import os

os.environ["SPARK_HOME"] = "C:\\spark-3.3.2-bin-hadoop3"
os.environ["HADOOP_HOME"] = "C:\\hadoop-3.4.1"
os.environ["JAVA_HOME"] = "C:\\jdk-11.0.26.4-hotspot"
os.environ["PATH"] += f";{os.environ['SPARK_HOME']}\\bin;{os.environ['HADOOP_HOME']}\\bin"

print("SPARK_HOME:", os.environ["SPARK_HOME"])
print("HADOOP_HOME:", os.environ["HADOOP_HOME"])
print("JAVA_HOME:", os.environ["JAVA_HOME"])


SPARK_HOME: C:\spark-3.3.2-bin-hadoop3
HADOOP_HOME: C:\hadoop-3.4.1
JAVA_HOME: C:\jdk-11.0.26.4-hotspot


In [4]:
from pyspark.sql import SparkSession
# Adjust SparkSession configuration to match your working setup
spark = SparkSession.builder \
    .master("spark://hadoop-master-node:7077") \
    .appName("SentimentDemo") \
    .config("spark.driver.memory", "512M") \
    .config("spark.executor.memory", "512M") \
    .config("spark.executor.cores", "2") \
    .config("spark.driver.host", "192.168.1.1") \
    .getOrCreate()

print("✅ Spark Session Created Successfully!")

✅ Spark Session Created Successfully!


### Process kaggle data to hadoop using spark

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv("../data/processed/twitter-kaggle-sentiment-cleaned.csv")

In [7]:
import pandas as pd
import pyspark

print("Pandas version:", pd.__version__)
print("PySpark version:", pyspark.__version__)


Pandas version: 1.5.3
PySpark version: 3.3.2


In [8]:
print(df.dtypes)


target           int64
id               int64
date            object
flag            object
user            object
text            object
cleaned_text    object
sentiment        int64
dtype: object


In [9]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, IntegerType

# Modify schema to use LongType for 'id' field
schema = StructType([
    StructField("target", IntegerType(), True),
    StructField("id", LongType(), True),  # Use LongType here for larger numbers
    StructField("date", StringType(), True),
    StructField("flag", StringType(), True),
    StructField("user", StringType(), True),
    StructField("text", StringType(), True),
    StructField("cleaned_text", StringType(), True),
    StructField("sentiment", IntegerType(), True)
])

# Create a Spark DataFrame from the cleaned Pandas DataFrame
spark_df = spark.createDataFrame(df, schema)


d:\Download\Anaconda\anaconda3\envs\sentiment-analysis-env\lib\site-packages\pyspark\sql\pandas\conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [10]:
# Write the Spark DataFrame to HDFS with the header
spark_df.write.mode("append").option("header", "true").csv("hdfs://hadoop-master-node:9000/sentiment/processed/twitter")


In [11]:
# Read the CSV files from HDFS, specifying that the first row contains the header
df = spark.read.csv("hdfs://hadoop-master-node:9000/sentiment/processed/twitter", header=True, inferSchema=True)

# Show the DataFrame to check the result
df.show()


+------+----------+--------------------+--------+---------------+--------------------+--------------------+---------+
|target|        id|                date|    flag|           user|                text|        cleaned_text|sentiment|
+------+----------+--------------------+--------+---------------+--------------------+--------------------+---------+
|     0|2200003196|Tue Jun 16 18:18:...|NO_QUERY|LaLaLindsey0609|@chrishasboobs AH...|        ahhh hope ok|       -1|
|     0|1467998485|Mon Apr 06 23:11:...|NO_QUERY|    sexygrneyes|@misstoriblack co...| cool tweet app razr|       -1|
|     0|2300048954|Tue Jun 23 13:40:...|NO_QUERY|     sammydearr|@TiannaChaos i kn...|know famili drama...|       -1|
|     0|1993474027|Mon Jun 01 10:26:...|NO_QUERY|    Lamb_Leanne|School email won'...|school email wont...|       -1|
|     0|2256550904|Sat Jun 20 12:56:...|NO_QUERY|    yogicerdito|upper airways pro...|upper airway problem|       -1|
|     0|2052380495|Sat Jun 06 00:32:...|NO_QUERY|      Y

In [13]:
df.count()

50000

In [ ]:
import praw
import pandas as pd
import time

In [ ]:
reddit = praw.Reddit(
    client_id = '-_LwwR2GAkBuc-mvHUVSEQ',
    client_secret = '5X25s7-NfqlEnlstXzdvHwWEKyyMfw',
    user_agent = 'Social Media Sentiment Analysis v1.0 by /u/Massive_Strategy75'
)

In [ ]:
def fetch_posts():
    posts = reddit.subreddit('WorldNews').search('world', limit = 100)
    # post_list = list(posts)
    return posts

In [ ]:
world_news_data = []

In [ ]:
posts = fetch_posts()
for post in posts:
    world_news_data.append({
        "title" : post.title,
        "score": post.score,
        "create_at": post.created_utc,
        "url" : post.url,
        "num_comments" : post.num_comments,
    })

In [ ]:
df_world_news = pd.DataFrame(world_news_data)
df_world_news

In [1]:
spark.stop()


NameError: name 'spark' is not defined